In [81]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import json
import os
import re
import urllib3
import urllib.parse
from urllib.request import urlopen
from urllib.parse import urlparse
import numpy as np

In [144]:
def get_status(logs):
    for log in logs:
        if log['message']:
            d = json.loads(log['message'])
            try:
                content_type = 'text/html' in d['message']['params']['response']['headers']['content-type']
                response_received = d['message']['method'] == 'Network.responseReceived'
                if content_type and response_received:
                    return d['message']['params']['response']['status']
            except:
                pass

def preprocess(line):
    return ' '.join(line.replace('\r',' ').replace('\xa0',' ').replace('\xad', ' ').strip().split())

def get_text(soup):
    text = []
    for script in soup(["script", "style","footer","input",'link','meta','button']):
        script.extract()

    paragraph = '\n'.join(list(dict.fromkeys([text for text in soup.stripped_strings])))
    return paragraph


final = '\n'.join(list(set(text)))

def get_all_links(url,link_hierarchy):
    child_links = []
    for script in soup(["script", "style"]):
        script.extract()
    
    sublinks = [a.get('href') for a in soup.find_all('a') if a.get('href') and a.get('href').startswith('/')]
    
    sublinks = [link[1:] for link in sublinks]
    sublinks = [*set(sublinks)]
    sublinks = list(filter(None, sublinks))
    
    for link in sublinks:
        if ".pdf" not in link:
            if 'www' in link:
                new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
            else:
                new_url = os.path.join(url[:url.rfind('//')+2], urlparse(url).netloc, link)
            child_links.append(new_url)
    
    link_hierarchy[url] = child_links
    return

In [26]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options = chrome_options)

In [121]:
links = input().split()
responses = []
selenium = []
status = 0
for link in links:
    try:
        page = requests.get(link)
        status = page.status_code
    except:
        pass
    if status == 200:
        page.encoding = 'utf-8'
        responses.append((link, page.text))
    else:
        page.encoding = 'utf-8'
        selenium.append(link)
        print(link, status, sep=' : ')
    status = 0

 https://alfabank.ru/ https://www.tinkoff.ru/ http://www.sberbank.ru/ru/person https://www.vtb.ru/ https://spbbank.ru/ https://business.rsb.ru/ https://www.gazprombank.ru/ https://www.raiffeisen.ru/ https://www.rosneft.ru/ https://lukoil.ru/ https://www.x5.ru/ru/ https://www.gazprom.ru/ https://www.gkm.ru/ https://www.tatneft.ru/ https://moskva.mts.ru/personal https://www.tplusgroup.ru/ https://www.evraz.com/ru/ https://www.eurochem.ru/


https://www.vtb.ru/ : 403
https://www.gazprombank.ru/ : 0
https://www.rosneft.ru/ : 403
https://lukoil.ru/ : 502
https://www.gkm.ru/ : 0
https://www.tatneft.ru/ : 0
https://www.tplusgroup.ru/ : 403


In [59]:
for link in selenium:
    driver.get(link)
    logs = driver.get_log('performance')
    print(link,' : ',get_status(logs))

https://www.vtb.ru/  :  200
https://www.gazprombank.ru/  :  200
https://www.rosneft.ru/  :  200
https://lukoil.ru/  :  None
https://www.gkm.ru/  :  200
https://www.tatneft.ru/  :  200
https://www.tplusgroup.ru/  :  200


In [208]:
soup = BeautifulSoup(page.text,'html.parser')